<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/augmentation/documented_augmentation/ate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textattack

In [10]:
import random
import timeit
import pickle

In [11]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

dataset = pd.read_json('/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/ATE_ABSITA_training_set/ate_absita_training.ndjson'
                       , lines=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
dataset.drop(columns=['id_sentence'], inplace=True)

In [13]:
print(f'Contains {len(dataset)} sentences')
dataset.head()

Contains 3054 sentences


,sentence,score,polarities,aspects_position,aspects
0,"Ottimo prodotto di marca, la qualità é veramen...",5,"[[0, 0], [0, 1], [1, 0]]","[[120, 142], [71, 79], [29, 36]]","[provvisto di una tasca, capiente, qualità]"
1,Ottimo rasoio dal semplice utilizzo. Rade molt...,5,"[[1, 0], [1, 0], [1, 0], [1, 0]]","[[18, 26], [37, 41], [79, 86], [99, 105]]","[semplice, Rade, Pratico, pulire]"
2,Un quarto delle dimensioni dello Show original...,5,"[[1, 0], [1, 0], [1, 0], [0, 0]]","[[118, 132], [51, 62], [65, 70], [16, 26]]","[modalità notte, prestazioni, suono, dimensioni]"
3,Il prodotto si presenta esattamente come in fo...,5,"[[1, 0], [1, 0], [1, 0]]","[[147, 158], [132, 140], [24, 48]]","[vestibilità, capienza, esattamente come in foto]"
4,"Superlativa, velocità in scrittura superiore a...",5,"[[1, 0]]","[[13, 21]]",[velocità]


###Sono presenti solo 20 duplicati. Si decide di non rimuoverli

In [21]:
len(dataset.groupby('sentence'))

3034

In [7]:
aug = dataset.copy()

##Augmentation Phase

###WordNet

In [22]:
from textattack.transformations import WordSwapWordNet

from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter
random.seed(33)


transformation = WordSwapWordNet(language='ita')
constraints = [RepeatModification(), StopwordModification()]

my_items = []
num_tras_to_add = 1

start_time = timeit.default_timer()
augmenter = Augmenter(transformation=transformation,
                      constraints=constraints,
                      pct_words_to_swap=1,
                      transformations_per_example=num_tras_to_add)

for row in aug.itertuples():
  result=augmenter.augment(row.sentence)
  # my_items.append({'sentence': row.sentence,
  #                  'score': row.score,
  #                  'polarities': row.polarities,
  #                  'aspects_position': row.aspects_position,
  #                  'aspects': row.aspects})
  for i in range(num_tras_to_add):
    ### QUI MODIFICHE
    my_items.append({'sentence': result[i],
                     'score': row.score,
                     'polarities': row.polarities,
                     'aspects_position': row.aspects_position,
                     'aspects': row.aspects})

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=aug.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=362.381112721
COLONNE:Index(['sentence', 'score', 'polarities', 'aspects_position', 'aspects'], dtype='object')
DIMENSIONE:(3054, 5)


In [23]:
augmeted_new.to_csv("ATE_sinonimi.csv",index=False)

In [24]:
augmeted_load=pd.read_csv('ATE_sinonimi.csv')  

###WordEmbedding

In [26]:
with open("/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/index2word.pkl", 'rb') as output:
  i2w = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/nn.pkl", 'rb') as output:
  nn_matrix = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

In [28]:
import textattack
#from TextAttack as T
embedding = textattack.shared.WordEmbedding(embedding_matrix,w2i,i2w,nn_matrix)

In [29]:
from typing import cast
t=cast(textattack.shared.AbstractWordEmbedding,embedding)

In [30]:
type(embedding)

textattack.shared.word_embedding.WordEmbedding

In [33]:
#from TextAttack import textattack

from textattack.transformations import WordSwapEmbedding
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter
random.seed(33)

transformation = WordSwapEmbedding(20,embedding)

constraints = [RepeatModification(), StopwordModification()]

my_items = []
num_tras_to_add = 1

start_time = timeit.default_timer()
augmenter = Augmenter(transformation=transformation,
                      constraints=constraints,
                      pct_words_to_swap=1,
                      transformations_per_example=num_tras_to_add)

for row in aug.head(10).itertuples():
  result=augmenter.augment(row.sentence)
  # my_items.append({'sentence': row.sentence,
  #                  'score': row.score,
  #                  'polarities': row.polarities,
  #                  'aspects_position': row.aspects_position,
  #                  'aspects': row.aspects})
  for i in range(num_tras_to_add):
    ### QUI MODIFICHE
    my_items.append({'sentence': result[i],
                     'score': row.score,
                     'polarities': row.polarities,
                     'aspects_position': row.aspects_position,
                     'aspects': row.aspects})

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new_embedding = pd.DataFrame(data=my_items, columns=aug.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=29.802634086999888
COLONNE:Index(['sentence', 'score', 'polarities', 'aspects_position', 'aspects'], dtype='object')
DIMENSIONE:(10, 5)


In [34]:
augmeted_new_embedding.to_csv("ATE_embedding.csv",index=False)

In [35]:
augmeted_load_embedding=pd.read_csv('ATE_embedding.csv')  